In [19]:
import pandas as pd
df = pd.read_csv('https://query.data.world/s/5lxj2ogcizom3m5unfcnlik52m7qie')
df.head(5)

,PUF_ID,sample,fpl,SWB_1,SWB_2,SWB_3,FWBscore,FWB1_1,FWB1_2,FWB1_3,...,PPMSACAT,PPREG4,PPREG9,PPT01,PPT25,PPT612,PPT1317,PPT18OV,PCTLT200FPL,finalwt
0,10350,2,3,5,5,6,55,3,3,3,...,1,4,8,0,0,0,0,1,0,0.367292
1,7740,1,3,6,6,6,51,2,2,3,...,1,2,3,0,0,0,0,2,0,1.327561
2,13699,1,3,4,3,4,49,3,3,3,...,1,4,9,0,0,0,1,2,1,0.835156
3,7267,1,3,6,6,6,49,3,3,3,...,1,3,7,0,0,0,0,1,0,1.410871
4,7375,1,3,4,4,4,49,3,3,3,...,1,2,4,0,0,1,0,4,1,4.260668


In [20]:
df.shape

(6394, 217)

In [21]:
df['SWB_1'].value_counts(normalize=True)
# put 6 and 7 together to get a column for ['satified_w_life']

 6    0.301220
 7    0.240069
 5    0.228026
 4    0.125586
 3    0.052393
 1    0.024085
 2    0.023772
-1    0.004692
-4    0.000156
Name: SWB_1, dtype: float64

Possible things to choose as target: SWB_1 = I am satisfied with my life, FWBscore = Financial well-being scale score. So I could either have the my ML problem be trying to predict one's satisfistion with life based on their responses to the CFPB financial well-being survey or one's financial well-being.

Choose your target. Which column in your tabular dataset will you predict?
 - My target will be trying to predict SWB_1 or I am satisfied with my life (also if curious on how I'm getting the names for these columns here's the link for the codebook https://data.world/cfpb/financial-well-being-survey-data/workspace/file?filename=cfpb_nfwbs-puf-codebook.pdf)
 Is your problem regression or classification?
 - Regression
  How is your target distributed?
  - There are technically 9 classes, but I'm going to use 6 and 7 as true and the rest could be treated as false. 
 Choose your evaluation metric(s).
 - R^2
  Choose which observations you will use to train, validate, and test your model.
- random split

In [22]:
df['satisfied_w_life'] = (df['SWB_1'] >= 6)

In [23]:
df['satisfied_w_life'].nunique()

2

In [27]:
df['satisfied_w_life'].value_counts(normalize=True)
# I have to beat 54% for my model to be any value

True     0.541289
False    0.458711
Name: satisfied_w_life, dtype: float64

In [43]:
from sklearn.model_selection import train_test_split

def wrangle(X):
    X = X.copy()
    
    unusable_variance = ['PUF_ID','sample']
    X = X.drop(columns = unusable_variance)
    
    return X

target = 'SWB_1'
train, test = train_test_split(df,test_size=0.05,random_state=42)

train, val = train_test_split(train,test_size=0.10,random_state=42)

train = wrangle(train)
val = wrangle(val)
test = wrangle(test)
print(train.shape,val.shape,test.shape)

X_train = train.drop(columns=target)
y_train = train[target]
X_val = val.drop(columns=target)
y_val = val[target]
X_test = test.drop(columns=target)
y_test = test[target]

(5466, 216) (608, 216) (320, 216)


In [40]:
import category_encoders as ce
from sklearn.impute import SimpleImputer
from sklearn.pipeline import make_pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

pipeline = make_pipeline(
ce.OrdinalEncoder(),
SimpleImputer(strategy='mean'),
RandomForestClassifier(n_estimators=100,random_state=42,n_jobs=-1))

pipeline.fit(X_train,y_train)

Pipeline(memory=None,
         steps=[('ordinalencoder',
                 OrdinalEncoder(cols=[], drop_invariant=False,
                                handle_missing='value', handle_unknown='value',
                                mapping=[], return_df=True, verbose=0)),
                ('simpleimputer',
                 SimpleImputer(add_indicator=False, copy=True, fill_value=None,
                               missing_values=nan, strategy='mean',
                               verbose=0)),
                ('randomforestclassifier',
                 RandomForestClassifier(bootstrap=True, class_weight=None,
                                        criterion='gini', max_depth=None,
                                        max_features='auto',
                                        max_leaf_nodes=None,
                                        min_impurity_decrease=0.0,
                                        min_impurity_split=None,
                                        min_samples_lea

In [44]:
# Change metric to R^2
y_pred = pipeline.predict(X_val)
print('Validation Accuracy', accuracy_score(y_val,y_pred))
y_pred = pipeline.predict(X_test)
print('Test Accuracy', accuracy_score(y_test,y_pred))
# check later with a graphic to see which feature is taking into account more than others

Validation Accuracy 0.6792763157894737
Test Accuracy 0.625
